In [28]:
%run "./utilspro.py"


In [25]:
# Unzipping the provided dataset
with zipfile.ZipFile("incident+management+process+enriched+event+log.zip", 'r') as z:
    # Listing files in the zip archive
    file_names = z.namelist()
    # Loading the dataset (assuming the first file in the archive is the desired dataset)
    dataset_path = z.extract(file_names[0])


# Loading the dataset
data = pd.read_csv(dataset_path)

# Displaying the first few rows of the dataset
data.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


## Data structure and summary

In [27]:
# Checking the data structure (data types and missing values)
data_info = data.info()

# Checking for placeholder values (assuming '?' is a placeholder)
placeholder_counts = data.apply(lambda x: x[x == '?'].count())

# Getting statistical summary of the dataset for numerical columns
data_summary = data.describe()

data_info, placeholder_counts, data_summary


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   caller_id                141712 non-null  object
 8   opened_by                141712 non-null  object
 9   opened_at                141712 non-null  object
 10  sys_created_by           141712 non-null  object
 11  sys_created_at           141712 non-null  object
 12  sys_updated_by           141712 non-null  object
 13  sys_updated_at           141712 non-null  object
 14  contact_type        

(None,
 number                          0
 incident_state                  0
 active                          0
 reassignment_count              0
 reopen_count                    0
 sys_mod_count                   0
 made_sla                        0
 caller_id                      29
 opened_by                    4835
 opened_at                       0
 sys_created_by              53076
 sys_created_at              53076
 sys_updated_by                  0
 sys_updated_at                  0
 contact_type                    0
 location                       76
 category                       78
 subcategory                   111
 u_symptom                   32964
 cmdb_ci                    141267
 impact                          0
 urgency                         0
 priority                        0
 assignment_group            14213
 assigned_to                 27496
 knowledge                       0
 u_priority_confirmation         0
 notify                          0
 problem_id  